<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [12]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [13]:
df = pd.DataFrame()
# (a)
df_list = []
for arch in archivos_anio:
  tmp = pd.read_csv(arch)
  tmp.columns = tmp.columns.str.lower()
  df_list.append(tmp)

df_anio = pd.concat(df_list, ignore_index= True)

# (b)

duplicados = df_codigos[df_codigos.duplicated('codigo_iso', keep=False)]
print("Códigos duplicados: \n", duplicados)

df_codigos = df_codigos.drop_duplicates(subset=['codigo_iso'], keep='first')

# (c)

df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
df.head()

Códigos duplicados: 
     codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo


,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [14]:
#Estructura del DataFrame
print("Filas y columnas:", df.shape)
print("\nNombres de columnas:")
print(df.columns.tolist())
print("\nTipos de datos:")
print(df.dtypes)
print("\nInformación general del DataFrame:")
print(df.info())

#Resumen estadístico
print("\nResumen estadístico:")
print(df.describe())

min_indice = df['indice'].min()
max_indice = df['indice'].max()
mean_indice = df['indice'].mean()

print("\nIndice -> Mínimo:", min_indice, "Máximo:", max_indice, "Promedio:", mean_indice)
print("\nPaís con menor indice (más libertad):")
print(df.loc[df['indice'] == min_indice, ['pais', 'anio', 'indice']])
print("\nPaís con mayor indice (menos libertad):")
print(df.loc[df['indice'] == max_indice, ['pais', 'anio', 'indice']])
print("\nPaís con mejor ranking (posición 1):")
print(df.loc[df['ranking'] == df['ranking'].min(), ['pais', 'anio', 'ranking']])
print("\nPaís con peor ranking:")
print(df.loc[df['ranking'] == df['ranking'].max(), ['pais', 'anio', 'ranking']])

#Datos faltantes
print("\nValores nulos por columna:")
print(df.isnull().sum())
print("\nProporción de valores faltantes por columna:")
print(df.isnull().mean())
print("\nColumnas con más del 30% de valores faltantes:")
print(df.columns[df.isnull().mean() > 0.3].tolist())

#Unicidad y duplicados
print("\nNúmero de países distintos:", df['pais'].nunique())
print("Número de años distintos:", df['anio'].nunique())
print("\nNúmero de filas duplicadas:", df.duplicated().sum())

#Validación cruzada de columnas
print("\nVerificación de código ISO único por país:")
print(df[['codigo_iso','pais']].drop_duplicates().duplicated('codigo_iso').sum())
duplicados = df[['codigo_iso','pais']].drop_duplicates()
problemas = duplicados[duplicados.duplicated('codigo_iso', keep=False)]
print("\nInconsistencias encontradas:")
print(problemas)

Filas y columnas: (3060, 5)

Nombres de columnas:
['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object

Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB
None

Resumen estadístico:
              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.0000




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [15]:
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

# (a)
resultados = []
for year in sorted(df_america['anio'].dropna().unique()):
    df_year = df_america[df_america['anio'] == year]
    if df_year['indice'].dropna().empty:
        continue

    pais_min = df_year.loc[df_year['indice'].idxmin(skipna=True), ['pais','indice']]
    pais_max = df_year.loc[df_year['indice'].idxmax(skipna=True), ['pais','indice']]

    resultados.append({
        'anio': year,
        'pais_mejor': pais_min['pais'],
        'indice_mejor': pais_min['indice'],
        'pais_peor': pais_max['pais'],
        'indice_peor': pais_max['indice']
    })

df_resultados_for = pd.DataFrame(resultados)
print(df_resultados_for.head())

# (b)
idx_min = df_america.groupby('anio')['indice'].idxmin(skipna=True)
idx_max = df_america.groupby('anio')['indice'].idxmax(skipna=True)

df_min = df_america.loc[idx_min.dropna(), ['anio','pais','indice']] \
                   .rename(columns={'pais':'pais_mejor','indice':'indice_mejor'})
df_max = df_america.loc[idx_max.dropna(), ['anio','pais','indice']] \
                   .rename(columns={'pais':'pais_peor','indice':'indice_peor'})

df_resultados_groupby = pd.merge(df_min, df_max, on='anio')
print(df_resultados_groupby.head())

   anio         pais_mejor  indice_mejor  pais_peor  indice_peor
0  2001             Canadá           0.8       Cuba        90.30
1  2002  Trinidad y Tobago           1.0       Cuba        97.83
2  2003  Trinidad y Tobago           2.0  Argentina     35826.00
3  2004  Trinidad y Tobago           2.0       Cuba        87.00
4  2005            Bolivia           4.5       Cuba        95.00
   anio         pais_mejor  indice_mejor  pais_peor  indice_peor
0  2001             Canadá           0.8       Cuba        90.30
1  2002  Trinidad y Tobago           1.0       Cuba        97.83
2  2003  Trinidad y Tobago           2.0  Argentina     35826.00
3  2004  Trinidad y Tobago           2.0       Cuba        87.00
4  2005            Bolivia           4.5       Cuba        95.00


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [16]:
pivot = pd.pivot_table(
    df_america,
    values='indice',
    index='pais',
    columns='anio',
    aggfunc='max',
    fill_value=0
)

print("Tabla dinámica (primeras filas):")
print(pivot.head())

# (a)
max_val = pivot.max().max()
min_val = pivot.replace(0, np.nan).min().min()
pais_max = pivot[pivot.eq(max_val).any(axis=1)].index.tolist()
pais_min = pivot[pivot.eq(min_val).any(axis=1)].index.tolist()
print(f"\n(a) País con mayor índice: {pais_max} (valor={max_val})")
print(f"(a) País con menor índice distinto de 0: {pais_min} (valor={min_val})")

# (b)
promedios_anio = pivot.mean(axis=0)
anio_max = promedios_anio.idxmax()
anio_min = promedios_anio.idxmin()
print(f"\n(b) Año con mayor promedio: {anio_max} (valor={promedios_anio.max()})")
print(f"(b) Año con menor promedio: {anio_min} (valor={promedios_anio.min()})")

# (c)
variabilidad = pivot.max(axis=1) - pivot.min(axis=1)
pais_var = variabilidad.idxmax()
print(f"\n(c) País con mayor variabilidad: {pais_var} (diferencia={variabilidad.max()})")

# (d)
constantes = pivot[(pivot.max(axis=1) == pivot.min(axis=1))].index.tolist()
print(f"\n(d) Países con índice constante en todos los años: {constantes}")

# (e)
sin_datos = pivot[(pivot == 0).all(axis=1)].index.tolist()
print(f"\n(e) Países sin datos: {sin_datos}")
print("Explicación: aparecen en la lista de 'america' pero no existen registros en el DataFrame original para esos países o años.")


Tabla dinámica (primeras filas):
anio               2001   2002     2003   2004   2005   2006   2007   2008  \
pais                                                                         
Antigua y Barbuda   0.0   0.00      0.0   0.00   0.00   0.00   0.00   0.00   
Argentina          12.0  15.17  35826.0  13.67  17.30  24.83  14.08  11.33   
Belize              0.0   0.00      0.0   0.00   0.00   0.00   0.00   0.00   
Bolivia            14.5   9.67     20.0   9.67   4.50  21.50  28.20  24.17   
Brasil             18.8  16.75     16.5  14.50  17.17  25.25  18.00  15.88   

anio                2009   2012   2013   2014   2015   2017   2018   2019  
pais                                                                       
Antigua y Barbuda   0.00   0.00  20.81   0.00   0.00   0.00   0.00   0.00  
Argentina          16.35  25.67  25.27  26.11  25.09  25.07  26.05  28.30  
Belize              0.00   0.00  17.05  18.54  20.61  23.43  24.55  27.50  
Bolivia            28.13  32.80  31.04  